## Data Offers Heatmap

In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives

The objective of this notebook recipe is to demonstrate how to compose and style a heatmap (a graphical representation of data where the individual values contained in a matrix are represented as colors). In this case the individual values consist of the total number of business intelligence and data analytics job offers published on a large employment web portal. 

### Data Preparation

Get the data and prepare the `Pandas.DataFrame` object.

In [2]:
conn = psycopg2.connect('dbname=jobsbg')
daily_df = pd.read_sql_query('SELECT * FROM data_offers.do_count_daily', conn, index_col='subm_date')
conn.close()

daily_df.index = pd.to_datetime(daily_df.index)
daily_df.head(5)

,subm_count
subm_date,
2017-09-27,4
2017-09-28,4
2017-09-29,2
2017-09-30,0
2017-10-01,0


#### Shaping the dataframe

Since I wanted to shape the heatmap in a way where the most active day of the week (Monday) is on the bottom, and the other days of the week following on top, I have to provide `Plotly` with a list of list with each of the lists being the same day (e.g. Monday) for the whole time period. In order to do that I first filled in the missing dates to create a complete rectangular dataframe table which could then be reshaped. See also: https://stackoverflow.com/a/45850005_

In [3]:
min_ts = min(daily_df.index)
max_ts = max(daily_df.index)
idx = pd.date_range(
    min_ts - dt.timedelta(days=min_ts.weekday()),
    max_ts + dt.timedelta(days=6-max_ts.weekday()))

In [4]:
daily_df = daily_df.reindex(idx)
daily_df.head()

,subm_count
2017-09-25,NaN
2017-09-26,NaN
2017-09-27,4.0
2017-09-28,4.0
2017-09-29,2.0


Now the dataframe can be reshaped into columns representing the days of the week, and then transposed to completely match our heatmap structure. That will make composing the chart itself very easy.

In [5]:
matrix_df_values = pd.DataFrame(daily_df.values.reshape(len(daily_df)//7, 7), columns=daily_df.index[:7].strftime('%A'))
matrix_df_values = matrix_df_values.T
matrix_df_values.head(7)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Monday,NaN,5.0,4.0,4.0,2.0,7.0,5.0,4.0,3.0,6.0,...,6.0,5.0,7.0,9.0,10.0,5.0,8.0,7.0,12.0,6.0
Tuesday,NaN,12.0,4.0,4.0,9.0,2.0,6.0,5.0,5.0,7.0,...,7.0,3.0,1.0,11.0,6.0,6.0,8.0,7.0,15.0,13.0
Wednesday,4.0,2.0,4.0,6.0,5.0,1.0,5.0,4.0,2.0,6.0,...,8.0,5.0,11.0,6.0,4.0,9.0,6.0,8.0,1.0,5.0
Thursday,4.0,7.0,2.0,4.0,4.0,3.0,3.0,2.0,2.0,2.0,...,9.0,12.0,7.0,9.0,5.0,7.0,10.0,10.0,6.0,NaN
Friday,2.0,7.0,1.0,8.0,5.0,3.0,4.0,5.0,9.0,9.0,...,8.0,10.0,13.0,9.0,13.0,9.0,11.0,2.0,8.0,NaN
Saturday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Sunday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [6]:
matrix_df_dates = daily_df
matrix_df_dates = matrix_df_dates.drop(['subm_count'], axis=1).reset_index()
matrix_df_dates = pd.DataFrame(matrix_df_dates.values.reshape(len(matrix_df_dates)//7, 7)).T
matrix_df_dates

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,2017-09-25,2017-10-02,2017-10-09,2017-10-16,2017-10-23,2017-10-30,2017-11-06,2017-11-13,2017-11-20,2017-11-27,...,2019-06-17,2019-06-24,2019-07-01,2019-07-08,2019-07-15,2019-07-22,2019-07-29,2019-08-05,2019-08-12,2019-08-19
1,2017-09-26,2017-10-03,2017-10-10,2017-10-17,2017-10-24,2017-10-31,2017-11-07,2017-11-14,2017-11-21,2017-11-28,...,2019-06-18,2019-06-25,2019-07-02,2019-07-09,2019-07-16,2019-07-23,2019-07-30,2019-08-06,2019-08-13,2019-08-20
2,2017-09-27,2017-10-04,2017-10-11,2017-10-18,2017-10-25,2017-11-01,2017-11-08,2017-11-15,2017-11-22,2017-11-29,...,2019-06-19,2019-06-26,2019-07-03,2019-07-10,2019-07-17,2019-07-24,2019-07-31,2019-08-07,2019-08-14,2019-08-21
3,2017-09-28,2017-10-05,2017-10-12,2017-10-19,2017-10-26,2017-11-02,2017-11-09,2017-11-16,2017-11-23,2017-11-30,...,2019-06-20,2019-06-27,2019-07-04,2019-07-11,2019-07-18,2019-07-25,2019-08-01,2019-08-08,2019-08-15,2019-08-22
4,2017-09-29,2017-10-06,2017-10-13,2017-10-20,2017-10-27,2017-11-03,2017-11-10,2017-11-17,2017-11-24,2017-12-01,...,2019-06-21,2019-06-28,2019-07-05,2019-07-12,2019-07-19,2019-07-26,2019-08-02,2019-08-09,2019-08-16,2019-08-23
5,2017-09-30,2017-10-07,2017-10-14,2017-10-21,2017-10-28,2017-11-04,2017-11-11,2017-11-18,2017-11-25,2017-12-02,...,2019-06-22,2019-06-29,2019-07-06,2019-07-13,2019-07-20,2019-07-27,2019-08-03,2019-08-10,2019-08-17,2019-08-24
6,2017-10-01,2017-10-08,2017-10-15,2017-10-22,2017-10-29,2017-11-05,2017-11-12,2017-11-19,2017-11-26,2017-12-03,...,2019-06-23,2019-06-30,2019-07-07,2019-07-14,2019-07-21,2019-07-28,2019-08-04,2019-08-11,2019-08-18,2019-08-25


#### Building the heatmap

The heatmap is composed of three dimensions: x, y, and z. The z dimension will hold our values matrix. Let's prepare that first:

In [7]:
cell_values = [
    [v for v in matrix_df_values.loc['Monday'].values],
    [v for v in matrix_df_values.loc['Tuesday'].values],
    [v for v in matrix_df_values.loc['Wednesday'].values],
    [v for v in matrix_df_values.loc['Thursday'].values],
    [v for v in matrix_df_values.loc['Friday'].values],
    [v for v in matrix_df_values.loc['Saturday'].values],
    [v for v in matrix_df_values.loc['Sunday'].values]
]

The approach to prepare the labels is the same:

In [8]:
cell_labels = [
    [str(v)[:10] for v in matrix_df_dates.iloc[0].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[1].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[2].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[3].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[4].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[5].values],
    [str(v)[:10] for v in matrix_df_dates.iloc[6].values]
]

A manually defined color scale to match with the presentation style:

In [9]:
colorscale =[
    [0.0, DS['colorramp']['acc1'][0]],
    [0.1111111111111111, DS['colorramp']['acc1'][1]],
    [0.2222222222222222, DS['colorramp']['acc1'][2]],
    [0.3333333333333333, DS['colorramp']['acc1'][3]],
    [0.4444444444444444, DS['colorramp']['acc1'][4]],
    [0.5555555555555556, DS['colorramp']['acc1'][5]],
    [0.6666666666666666, DS['colorramp']['acc1'][6]],
    [0.7777777777777778, DS['colorramp']['acc1'][7]],
    [0.8888888888888888, DS['colorramp']['acc1'][8]],
    [1.0, DS['colorramp']['acc1'][9]]
]

Finally, the heatmap trace definition:

In [10]:
hm_trace = go.Heatmap(
    x=[w for w in daily_df.index[::7]],
    y=[d for d in matrix_df_values.index],
    z=cell_values,
    text=cell_labels,
    hoverinfo='text+z+y',
    colorscale=colorscale,
    showscale=False,
    xgap=0,
    ygap=0,
)

data = [hm_trace]

Layout preparation is fairly standard. Labels have been turned off for both axes.

In [11]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Data Jobs Daily Heatmap',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    autosize=True,
    showlegend=False,
    hidesources=True,
    yaxis=dict(),
)

In [12]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='data_jobs_subm_heatmap.html')

In [13]:
# Uncomment the line below to export an HTML version of the chart.
plotly.offline.plot(fig, filename='data_offers_subm_heatmap.html', show_link=False)

'file:///games/WORKSPACE/jpynb_Job_Market_Trends_Bulgaria/workbooks/data_offers_subm_heatmap.html'

In [14]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)